In [1]:
from secret_key import openapi_key
import os

os.environ['OPENAI_API_KEY'] = openapi_key

#print(openapi_key)
print(os.environ['OPENAI_API_KEY'])

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains.llm import LLMChain
import json


model = OllamaLLM(model="llama3.2:1b", temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables= ['cuisine'],
    template = "I want to open an restaurent for {cuisine} food. Suggest a fancy name. Return only one name without any other text"
)

prompt_template_name.format(cuisine="Mexican")

name_chain = LLMChain(llm=model, prompt=prompt_template_name)

o = name_chain.invoke("Gujarati")

outjson = json.loads( json.dumps(o))

print( "generated name: " + outjson['text'] ) 

generated name: "Khazana Khaas"


In [8]:
prompt_template_items = PromptTemplate(
    input_variables= ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list"
)

food_items_chain = LLMChain(llm=model, prompt=prompt_template_items)


In [9]:
from langchain.chains.sequential import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain] )

response = chain.run("Indian")
print( response )


Here are some menu item suggestions for Spicehaven:

Spicy Shrimp and Chorizo Paella, Currywurst Burger with Spicy Mayo, Jamaican Jerk Chicken Tenders with Mango Slaw, Ghost Pepper Wings with Blue Cheese Dip, Thai Green Curry Risotto Bowl with Shrimp and Vegetables, Indian Butter Chicken Naan Bites with Cucumber Raita, Korean BBQ Beef Tacos with Kimchi Slaw, Sri Lankan Fish Ambulthiyal (Sour Soup) with Crusty Bread, Jamaican Jerk Pork Belly Buns with Spicy Pickle Relish.


In [10]:
prompt_template_name2 = PromptTemplate(
    input_variables= ['cuisine'],
    template = "I want to open an restaurent for {cuisine} food. Suggest a fancy name. Return only one name without any other text"
)

prompt_template_name2.format(cuisine="Mexican")

name_chain2 = LLMChain(llm=model, prompt=prompt_template_name2, output_key="restaurant_name")

prompt_template_items2 = PromptTemplate(
    input_variables= ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list"
)

food_items_chain2 = LLMChain(llm=model, prompt=prompt_template_items2, output_key="menu_items")



In [13]:
from langchain.chains.sequential import SequentialChain

chain = SequentialChain(
    chains = [name_chain2, food_items_chain2],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items' ]
)

chain.invoke({'cuisine':'arabic'})

{'cuisine': 'arabic',
 'restaurant_name': '"Al-Furqan Al-Khaleesi"',
 'menu_items': 'Here are some menu item suggestions for "Al-Furqan Al-Khaleesi" (The Golden Standard of Hospitality):\n\nMeat dishes: lamb koftas, chicken shawarma, grilled fish with tahini sauce, kebabs of beef or lamb, shwarma-style lamb burgers.\n\nVegetarian options: falafel wraps, grilled eggplant parmesan, roasted vegetables with hummus, stuffed bell peppers, tabbouleh salad.\n\nSides and accompaniments: pickled turnips, preserved lemon, olives, flatbread with olive oil and herbs, grilled pita bread with tzatziki sauce.\n\nDesserts: baklava, ma\'amoul cookies, fresh fruit platter with honey and pistachios.'}